In [ ]:
!pip install contextualized-topic-models==2.2.0

## Import General Utility Libraries

In [ ]:
import re
import urllib
import gzip
import io
import csv
import random
from collections import defaultdict
from tqdm import tqdm

Where to store the data file. If you want, you can adjust the path.

In [ ]:
path_before_1990 = '/content/drive/My Drive/titles_before_1990.txt'
path_from_1990_to_2009 = '/content/drive/My Drive/titles_from_1990_to_2009.txt'
path_from_2010 = '/content/drive/My Drive/titles_from_2010.txt'

Execute the following cell only once to download the data and write it as a file to your google drive. Afterwards, skip this cell or comment it out.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# to download the data manually or get more information, go to: https://dblp.org/faq/How+can+I+download+the+whole+dblp+dataset.html
url = 'https://dblp.uni-trier.de/xml/dblp.xml.gz'
# num_titles = 500000  # the (max)number of titles to load


def load_gzip_file(url):
    """Download Gzip-file."""
    response = urllib.request.urlopen(url)
    compressed_file = io.BytesIO(response.read())
    decompressed_file = gzip.GzipFile(fileobj=compressed_file)
    return decompressed_file

def extract_titles(input_file, max_num=40000):
    """Extract title and publication year of dblp papers, given as input file.

    Divide the papers into 3 time periods.

    Collect max max_num papers per time period.
    """
    pairs_before_1990 = []
    count_before_1990 = 0
    pairs_from_1990_to_2009 = []
    count_from_1990_to_2009 = 0
    pairs_from_2010 = []
    count_from_2010 = 0
    got_title = False
    for line in tqdm(input_file):
        line_str = line.decode('utf-8')
        if got_title:
            # we have a title and check for the corresponding year
            year_result = re.search(r'<year>(.*)</year>', line_str)
            if year_result:
                # we also have the year and thus save the title-year pair
                year = int(year_result.group(1))
                if year < 1990:
                    pairs_before_1990.append((title, year))
                    count_before_1990 += 1
                elif year < 2010:
                    pairs_from_1990_to_2009.append((title, year))
                    count_from_1990_to_2009 += 1
                else:
                    pairs_from_2010.append((title, year))
                    count_from_2010 += 1
                got_title = False
        else:
            # we have no title and search for title
            result = re.search(r'<title>(.*)</title>', line_str)
            if result:
                title = result.group(1)
                if len(title.split(' ')) < 3:
                    # only include titles with at least four words
                    continue
                got_title = True

        if count_before_1990 >= max_num and count_from_1990_to_2009 >= max_num and count_from_2010 >= max_num:
            return pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010

    return pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010

def save_data(pairs, file_path):
    with open(file_path, 'w') as fout:
        writer = csv.writer(fout)
        for pair in pairs:
            writer.writerow(pair)

in_file = load_gzip_file(url)
pairs_before_1990, pairs_from_1990_to_2009, pairs_from_2010 = extract_titles(in_file)
save_data(pairs_before_1990, path_before_1990)
save_data(pairs_from_1990_to_2009, path_from_1990_to_2009)
save_data(pairs_from_2010, path_from_2010)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


16410690it [00:38, 424474.45it/s]


Mount your google drive (in case it is not yet mounted) so that the newly created files are available.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# LDA

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

num_lda_topics = 5

### Before the 1990s:

In [ ]:
with open(path_before_1990) as fin:
    reader = csv.reader(fin)
    titles = [row[0] for row in reader][:10000]

Let's perform some simple preprocessing:

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords_eng = set(stopwords.words('english'))
stopwords_de = set(stopwords.words('german'))

In [ ]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z ]', '', text)
    text = text.lower()
    word_tokens = word_tokenize(text)
    filtered_sentence = []
    for w in word_tokens:
      if (w not in stopwords_eng) and (w not in stopwords_de):
        filtered_sentence.append(w)
    output = " ".join(filtered_sentence)
    return output

prepro_titles = [preprocess_text(title) for title in titles]

In [ ]:
prepro_titles[:10]

['object model capabilities distributed object management',
 'distributed object management technology',
 'muffin distributed database machine',
 'algebraical optimization ftaexpressions',
 'wissensrepraumlsentation maschinelles lernen',
 'algebraic characterization stuf',
 'systemarchitektur lilog',
 'mengenorientierte auswertung anfragen logikprogrammiersprache prolog',
 'definite resolution constraint languages',
 'dokumentation syntax liloggrammatik']

Now we turn the documents (or titles in this case) into a matrix feature representation.

In [ ]:
num_features = 10000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=num_features, stop_words='english')
tf = tf_vectorizer.fit_transform(prepro_titles)
tf_feature_names = tf_vectorizer.get_feature_names_out()

In [ ]:
lda = LatentDirichletAllocation(n_components=num_lda_topics, max_iter=5, learning_method='online', random_state=42).fit(tf)

In [ ]:
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {topic_idx}:', end=' ')
    print(' '.join([tf_feature_names[i] for i in topic.argsort()[:-12 - 1:-1]]))

Topic 0: systems linear languages method theorem general decision automata architecture theories simple support
Topic 1: algorithm complexity software problems equations finite solution problem time fortran number search
Topic 2: note design sets problem implementation optimal finding optimization remark algebraic retrieval trees
Topic 3: data systems theory programming parallel computer model analysis algorithms using control language
Topic 4: information logic programs functions set efficient grammars new structures discrete matrix programming


Topics:
0. Memory-Optimized Parallel Models
1. ...
2. ...

### From 1990 to 2009:

Add your code for topic modelling the period from 1990 to 2009 here...

In [ ]:
with open(path_from_1990_to_2009) as fin:
    reader = csv.reader(fin)
    titles = [row[0] for row in reader][:10000]

In [ ]:
prepro_titles = [preprocess_text(title) for title in titles]

In [ ]:
num_features = 10000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=num_features, stop_words='english')
tf = tf_vectorizer.fit_transform(prepro_titles)
tf_feature_names = tf_vectorizer.get_feature_names_out()

In [ ]:
lda = LatentDirichletAllocation(n_components=num_lda_topics, max_iter=5, learning_method='online', random_state=42).fit(tf)

In [ ]:
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {topic_idx}:', end=' ')
    print(' '.join([tf_feature_names[i] for i in topic.argsort()[:-12 - 1:-1]]))

Topic 0: design theory nonlinear optimal algorithms problems optimization equilibrium chemical application method continuous
Topic 1: networks using modeling dynamic scheduling approach query batch algorithm distributed engineering search
Topic 2: data control model optimization methods multimedia video applications uncertainty selection databases global
Topic 3: systems information analysis web network based games problem spatial security wireless knowledge
Topic 4: process models mining processes management framework processing dynamics computing image recognition linear


Topics:
0. Memory-Optimized Parallel Models
1. ...
2. ...

### From 2010 onwards:

Add your code for topic modelling the period from 2010 onwards here...

In [ ]:
with open(path_from_2010) as fin:
    reader = csv.reader(fin)
    titles = [row[0] for row in reader][:10000]

In [ ]:
prepro_titles = [preprocess_text(title) for title in titles]

In [ ]:
num_features = 10000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=num_features, stop_words='english')
tf = tf_vectorizer.fit_transform(prepro_titles)
tf_feature_names = tf_vectorizer.get_feature_names_out()

In [ ]:
lda = LatentDirichletAllocation(n_components=num_lda_topics, max_iter=5, learning_method='online', random_state=42).fit(tf)

In [ ]:
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {topic_idx}:', end=' ')
    print(' '.join([tf_feature_names[i] for i in topic.argsort()[:-12 - 1:-1]]))

Topic 0: analysis management information models human dynamic risk framework public parallel language perspective
Topic 1: systems data engineering mining algorithms using web privacy technologies recommender development issues
Topic 2: networks control design model online processing models databases overview query students integrity
Topic 3: social research ethical network technology spatial education case methods applications detection scientific
Topic 4: data ethics learning science knowledge theory distributed search big approach retrieval statistical


Topics:
0. Memory-Optimized Parallel Models
1. ...
2. ...

# Combined Topic Models

Method developed by [Bianchi et al. 2021](https://aclanthology.org/2021.acl-short.96/).

[A 6min presentation of the paper by one of the authors.](https://underline.io/lecture/25716-pre-training-is-a-hot-topic-contextualized-document-embeddings-improve-topic-coherence)

Code: [https://github.com/MilaNLProc/contextualized-topic-models](https://github.com/MilaNLProc/contextualized-topic-models)

Tutorial: [https://colab.research.google.com/drive/1fXJjr_rwqvpp1IdNQ4dxqN4Dp88cxO97?usp=sharing](https://colab.research.google.com/drive/1fXJjr_rwqvpp1IdNQ4dxqN4Dp88cxO97?usp=sharing)

Again, perform topic modelling for the three time periods - this time using the combined topic models (CTMs).

You can use and adapt the code from the tutorial linked above.

Use the available GPU for faster running times.

In [ ]:
!pip install contextualized-topic-models==2.5.0

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing

num_ctm_topics = 5  # you can also choose a higher number of topics

### Before the 1990s:

In [ ]:
with open(path_before_1990) as fin:
    reader = csv.reader(fin)
    documents = [row[0] for row in reader][:10000]

In [ ]:
sp = WhiteSpacePreprocessing(documents, "english")
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

In [ ]:
qt = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = qt.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
ctm = CombinedTM(bow_size=len(qt.vocab), contextual_size=768, n_components=num_ctm_topics) # n_components = topics
ctm.fit(training_dataset) # run the model

In [ ]:
ctm.get_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


defaultdict(list,
            {0: ['optimisation',
              'event',
              'flowchart',
              'transportable',
              'organizing',
              'macro',
              'spelling',
              'controlling',
              'vision',
              'paradigms'],
             1: ['automata',
              'logic',
              'sup',
              'theory',
              'sub',
              'free',
              'sets',
              'grammars',
              'context',
              'theorem'],
             2: ['algorithm',
              'algorithms',
              'linear',
              'equations',
              'problem',
              'solution',
              'optimal',
              'parallel',
              'time',
              'problems'],
             3: ['programming',
              'language',
              'system',
              'data',
              'design',
              'object',
              'oriented',
              'based',
          

### From 1990 to 2009

In [ ]:
with open(path_from_1990_to_2009) as fin:
    reader = csv.reader(fin)
    documents = [row[0] for row in reader][:10000]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
sp = WhiteSpacePreprocessing(documents, "english")
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

In [ ]:
qt = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = qt.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
ctm = CombinedTM(bow_size=len(qt.vocab), contextual_size=768, n_components=num_ctm_topics) # n_components = topics
ctm.fit(training_dataset) # run the model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Batches:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch: [100/100]	 Seen Samples: [992000/992600]	Train Loss: 31.450843072706654	Time: 0:00:03.242955: : 100it [05:34,  3.35s/it]
100%|██████████| 156/156 [00:03<00:00, 41.25it/s]


In [ ]:
ctm.get_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


defaultdict(list,
            {0: ['mining',
              'data',
              'recognition',
              'spatial',
              'query',
              'temporal',
              'image',
              'video',
              'indexing',
              'face'],
             1: ['information',
              'multimedia',
              'mobile',
              'wireless',
              'web',
              'security',
              'management',
              'portals',
              'services',
              'networks'],
             2: ['decisions',
              'pca',
              'inventory',
              'practice',
              'timed',
              'queueing',
              'continuity',
              'chip',
              'mimo',
              'expert'],
             3: ['problems',
              'programming',
              'methods',
              'algorithms',
              'problem',
              'global',
              'theory',
              'equilibrium',
         

### From 2010 onwards

In [ ]:
with open(path_from_2010) as fin:
    reader = csv.reader(fin)
    documents = [row[0] for row in reader][:10000]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
sp = WhiteSpacePreprocessing(documents, "english")
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/contextualized_topic_models/utils/preprocessing.py:24: UserWarning: WhiteSpacePreprocessing is deprecated and will be removed in future versions.Use WhiteSpacePreprocessingStopwords.
  warnings.warn("WhiteSpacePreprocessing is deprecated and will be removed in future versions."


In [ ]:
qt = TopicModelDataPreparation("all-mpnet-base-v2")
training_dataset = qt.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
ctm = CombinedTM(bow_size=len(qt.vocab), contextual_size=768, n_components=num_ctm_topics) # n_components = topics
ctm.fit(training_dataset) # run the model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Batches:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch: [100/100]	 Seen Samples: [985600/991500]	Train Loss: 29.037352883970584	Time: 0:00:02.799553: : 100it [05:27,  3.28s/it]
100%|██████████| 155/155 [00:03<00:00, 40.84it/s]


In [ ]:
ctm.get_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


defaultdict(list,
            {0: ['company',
              'involved',
              'layered',
              'members',
              'dissemination',
              'intensive',
              'goods',
              'transforming',
              'estimating',
              'communicating'],
             1: ['spatial',
              'data',
              'mining',
              'web',
              'processing',
              'query',
              'retrieval',
              'text',
              'databases',
              'semantic'],
             2: ['social',
              'networks',
              'analysis',
              'network',
              'management',
              'security',
              'knowledge',
              'online',
              'privacy',
              'media'],
             3: ['research',
              'science',
              'engineering',
              'ethics',
              'ethical',
              'human',
              'education',
              'tec